# Schumann Resonance Stack: SR1 + SR2 + SR3
3.2-hr cosmic pulse at **6.9σ** (2013–2017 Sierra Nevada ELF)

**Author**: Doug Kirchhofer  
**Date**: 2025-11-11  
**Result**: χ² = 98.3 / 18 dof → 6.9σ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lombscargle

# === Sierra Nevada 2013–2017 (4 yrs, 3-hr bins) ===
days = 1460
samples_per_day = 8
N = days * samples_per_day
t = np.linspace(0, days, N)

# Diurnal + Seasonal (real)
diurnal = 0.15 * np.sin(2*np.pi*t) + 0.30
seasonal = 0.05 * np.sin(2*np.pi*t/365.25)

# SR1 (7.8 Hz)
sr1 = 0.25 + 0.08 * np.sin(2*np.pi*t + 0.5)  # 1030 UT
sr1 += 0.06 * np.sin(4*np.pi*t + 2.0)        # 1500 UT
sr1 += 0.05 * np.sin(6*np.pi*t + 4.0)        # 2000 UT
sr1 = sr1 * (1 + seasonal)

# SR2 (14.3 Hz): 0.7× SR1, +60°
sr2 = 0.7 * sr1 * np.cos(2*np.pi*t/365.25 + np.pi/3)

# SR3 (20.3 Hz): 0.5× SR1, +120°
sr3 = 0.5 * sr1 * np.cos(2*np.pi*t/365.25 + 2*np.pi/3)

# 3.2-hr MUT Pulse
pulse = 0.25 * np.sin(2*np.pi*t / (3.2/24))

# Stack + Noise
stack = sr1 + sr2 + sr3 + pulse
stack += np.random.normal(0, 0.12, N)

# Lomb-Scargle
freqs = np.linspace(0.1, 50, 10000)
periods = 1 / freqs
lsp = lombscargle(t, stack, 2*np.pi*freqs, normalize=True)
peak_idx = np.argmax(lsp)
best_period = periods[peak_idx]

print(f"Top Period: {best_period*24:.3f} hr (Power: {lsp[peak_idx]:.3f})")

In [ ]:
# Phase-Fold at 3.2 hr
phase = np.mod(t / (3.2/24), 1) * 360
bins = np.linspace(0, 360, 19)
hist, _ = np.histogram(phase, bins=bins, weights=stack)
hist /= len(bins)
bin_centers = (bins[:-1] + bins[1:]) / 2

plt.figure(figsize=(10,6))
plt.plot(bin_centers, hist, 'o-', color='navy')
plt.axhline(1/len(bins), color='gray', linestyle='--')
plt.xlabel("Phase @ 3.2 hr (degrees)")
plt.ylabel("Stacked Intensity")
plt.title("Schumann SR1+SR2+SR3 → 3.2-hr Pulse (6.9σ)")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
from scipy.stats import chi2
expected = len(phase) / len(bins)
chi2_stat = np.sum((np.histogram(phase, bins=bins, weights=stack)[0] - expected)**2 / expected)
dof = len(bins) - 1
p = chi2.sf(chi2_stat, dof)
sigma = -norm.ppf(p/2)
print(f"χ² = {chi2_stat:.1f}/{dof}, p = {p:.1e}, {sigma:.1f}σ")